## CBoW மாடலை பயிற்சி செய்வது

இந்த நோட்புக் [AI for Beginners Curriculum](http://aka.ms/ai-beginners) இன் ஒரு பகுதியாகும்.

இந்த உதாரணத்தில், நாங்கள் CBoW மொழி மாடலை பயிற்சி செய்து, எங்கள் சொந்த Word2Vec எம்பெடிங் இடத்தை பெறப் பார்க்கிறோம். AG News தரவுத்தொகுப்பை உரையின் மூலமாக பயன்படுத்துவோம்.


In [ ]:
import torch
import torchtext
import os
import collections
import builtins
import random
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

முதலில் நமது தரவுத்தொகுப்பை ஏற்றவும், டோக்கனைசர் மற்றும் சொற்களஞ்சியத்தை வரையறுக்கவும். கணக்கீடுகளை கொஞ்சம் வரையறுக்க `vocab_size`-ஐ 5000 ஆக அமைப்போம்.


In [ ]:
def load_dataset(ngrams = 1, min_freq = 1, vocab_size = 5000 , lines_cnt = 500):
    tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
    print("Loading dataset...")
    test_dataset, train_dataset  = torchtext.datasets.AG_NEWS(root='./data')
    train_dataset = list(train_dataset)
    test_dataset = list(test_dataset)
    classes = ['World', 'Sports', 'Business', 'Sci/Tech']
    print('Building vocab...')
    counter = collections.Counter()
    for i, (_, line) in enumerate(train_dataset):
        counter.update(torchtext.data.utils.ngrams_iterator(tokenizer(line),ngrams=ngrams))
        if i == lines_cnt:
            break
    vocab = torchtext.vocab.Vocab(collections.Counter(dict(counter.most_common(vocab_size))), min_freq=min_freq)
    return train_dataset, test_dataset, classes, vocab, tokenizer

In [ ]:
train_dataset, test_dataset, _, vocab, tokenizer = load_dataset()

Loading dataset...
Building vocab...


In [ ]:
def encode(x, vocabulary, tokenizer = tokenizer):
    return [vocabulary[s] for s in tokenizer(x)]

## CBoW மாடல்

CBoW $2N$ அண்டை வார்த்தைகளை அடிப்படையாகக் கொண்டு ஒரு வார்த்தையை கணிக்க கற்றுக்கொள்கிறது. உதாரணமாக, $N=1$ என்றால், *I like to train networks* என்ற வாக்கியத்திலிருந்து பின்வரும் ஜோடிகளை பெறுவோம்: (like,I), (I, like), (to, like), (like,to), (train,to), (to, train), (networks, train), (train,networks). இங்கு, முதல் வார்த்தை உள்ளீடாக பயன்படுத்தப்படும் அண்டை வார்த்தை, இரண்டாவது வார்த்தை நாம் கணிக்க முயற்சிக்கும் வார்த்தை.

அடுத்த வார்த்தையை கணிக்க ஒரு நெட்வொர்க் உருவாக்க, அண்டை வார்த்தையை உள்ளீடாக வழங்க வேண்டும், மற்றும் வார்த்தை எண்ணை வெளியீடாக பெற வேண்டும். CBoW நெட்வொர்க் அமைப்பு பின்வருமாறு உள்ளது:

* உள்ளீடு வார்த்தை எம்பெடிங் லேயர் வழியாக அனுப்பப்படும். இந்த எம்பெடிங் லேயர் தான் நமது Word2Vec எம்பெடிங் ஆக இருக்கும், எனவே இதை `embedder` மாறியாக தனியாக வரையறுக்கிறோம். இந்த எடுத்துக்காட்டில் எம்பெடிங் அளவு = 30 ஆக பயன்படுத்துவோம், ஆனால் நீங்கள் அதிக பரிமாணங்களுடன் (உண்மையான word2vec 300) பரிசோதிக்க விரும்பலாம்.
* எம்பெடிங் வெக்டர் பின்னர் ஒரு லினியர் லேயருக்கு அனுப்பப்படும், இது வெளியீடு வார்த்தையை கணிக்கிறது. எனவே இதற்கு `vocab_size` நியூரான்கள் இருக்கும்.

வெளியீட்டிற்காக, `CrossEntropyLoss` ஐ இழப்பீட்டு செயல்பாடாக பயன்படுத்தினால், எதிர்பார்க்கப்படும் முடிவுகளை ஒரே-ஹாட் என்கோடிங் இல்லாமல், வெறும் வார்த்தை எண்களாக வழங்க வேண்டும்.


In [ ]:
vocab_size = len(vocab)

embedder = torch.nn.Embedding(num_embeddings = vocab_size, embedding_dim = 30)
model = torch.nn.Sequential(
    embedder,
    torch.nn.Linear(in_features = 30, out_features = vocab_size),
)

print(model)

Sequential(
  (0): Embedding(5002, 30)
  (1): Linear(in_features=30, out_features=5002, bias=True)
)


## பயிற்சி தரவுகளை தயாரித்தல்

இப்போது, CBoW வார்த்தை ஜோடிகளை உரையிலிருந்து கணக்கிடும் முக்கிய செயல்பாட்டை நிரலிடுவோம். இந்த செயல்பாடு சாளர அளவை குறிப்பிட அனுமதிக்கும், மேலும் ஒரு ஜோடி தொகுப்பை - உள்ளீடு மற்றும் வெளியீடு வார்த்தையை - திருப்பி அனுப்பும். இந்த செயல்பாடு வார்த்தைகளில் மட்டுமல்லாமல், வெக்டர்கள்/டென்சர்களிலும் பயன்படுத்தப்படலாம் - இது உரையை குறியாக்க அனுமதிக்கும், அதை `to_cbow` செயல்பாட்டிற்கு அனுப்புவதற்கு முன்.


In [ ]:
def to_cbow(sent,window_size=2):
    res = []
    for i,x in enumerate(sent):
        for j in range(max(0,i-window_size),min(i+window_size+1,len(sent))):
            if i!=j:
                res.append([sent[j],x])
    return res

print(to_cbow(['I','like','to','train','networks']))
print(to_cbow(encode('I like to train networks', vocab)))

[['like', 'I'], ['to', 'I'], ['I', 'like'], ['to', 'like'], ['train', 'like'], ['I', 'to'], ['like', 'to'], ['train', 'to'], ['networks', 'to'], ['like', 'train'], ['to', 'train'], ['networks', 'train'], ['to', 'networks'], ['train', 'networks']]
[[232, 172], [5, 172], [172, 232], [5, 232], [0, 232], [172, 5], [232, 5], [0, 5], [1202, 5], [232, 0], [5, 0], [1202, 0], [5, 1202], [0, 1202]]


பயிற்சி தரவுத்தொகுப்பை தயாரிப்போம். அனைத்து செய்திகளையும் ஆய்வு செய்து, `to_cbow` ஐ அழைத்து சொற்களின் ஜோடிகளின் பட்டியலை பெறுவோம், மற்றும் அந்த ஜோடிகளை `X` மற்றும் `Y` இல் சேர்ப்போம். நேரத்தை மிச்சப்படுத்துவதற்காக, முதல் 10k செய்தி உருப்படிகளை மட்டுமே கருதுவோம் - நீங்கள் அதிக நேரம் காத்திருக்க விரும்பினால், மற்றும் சிறந்த எம்பெடிங்குகளை பெற விரும்பினால், இந்த வரம்பை எளிதாக நீக்கலாம் :)


In [ ]:
X = []
Y = []
for i, x in zip(range(10000), train_dataset):
    for w1, w2 in to_cbow(encode(x[1], vocab), window_size = 5):
        X.append(w1)
        Y.append(w2)

X = torch.tensor(X)
Y = torch.tensor(Y)

நாங்கள் அந்த தரவையும் ஒரு தரவுத்தொகுப்பாக மாற்றி, டேட்டா லோடரை உருவாக்குவோம்:


In [ ]:
class SimpleIterableDataset(torch.utils.data.IterableDataset):
    def __init__(self, X, Y):
        super(SimpleIterableDataset).__init__()
        self.data = []
        for i in range(len(X)):
            self.data.append( (Y[i], X[i]) )
        random.shuffle(self.data)

    def __iter__(self):
        return iter(self.data)

நாம் அந்த தரவையும் ஒரு தரவுத்தொகுப்பாக மாற்றி, தரவுதிரவியை உருவாக்குவோம்:


In [ ]:
ds = SimpleIterableDataset(X, Y)
dl = torch.utils.data.DataLoader(ds, batch_size = 256)

இப்போது நிஜமான பயிற்சியை தொடங்குவோம். நாம் `SGD` ஆப்டிமைசரை மிகவும் அதிகமான கற்றல் விகிதத்துடன் பயன்படுத்துவோம். நீங்கள் `Adam` போன்ற பிற ஆப்டிமைசர்களுடன் முயற்சி செய்து பார்க்கலாம். ஆரம்பத்தில் 10 எபோக்களுக்காக பயிற்சி செய்வோம் - மேலும் குறைந்த இழப்பை விரும்பினால், இந்த செல்களை மீண்டும் இயக்கலாம்.


In [ ]:
def train_epoch(net, dataloader, lr = 0.01, optimizer = None, loss_fn = torch.nn.CrossEntropyLoss(), epochs = None, report_freq = 1):
    optimizer = optimizer or torch.optim.Adam(net.parameters(), lr = lr)
    loss_fn = loss_fn.to(device)
    net.train()

    for i in range(epochs):
        total_loss, j = 0, 0, 
        for labels, features in dataloader:
            optimizer.zero_grad()
            features, labels = features.to(device), labels.to(device)
            out = net(features)
            loss = loss_fn(out, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss
            j += 1
        if i % report_freq == 0:
            print(f"Epoch: {i+1}: loss={total_loss.item()/j}")

    return total_loss.item()/j

In [ ]:
train_epoch(net = model, dataloader = dl, optimizer = torch.optim.SGD(model.parameters(), lr = 0.1), loss_fn = torch.nn.CrossEntropyLoss(), epochs = 10)

Epoch: 1: loss=5.664632366860172
Epoch: 2: loss=5.632101973960962
Epoch: 3: loss=5.610399051405015
Epoch: 4: loss=5.594621561080262
Epoch: 5: loss=5.582538017415446
Epoch: 6: loss=5.572900234519603
Epoch: 7: loss=5.564951676341915
Epoch: 8: loss=5.558288112064614
Epoch: 9: loss=5.552576955031129
Epoch: 10: loss=5.547634165194347


5.547634165194347

## Word2Vec ஐ முயற்சிக்கிறோம்

Word2Vec ஐ பயன்படுத்த, நமது சொற்களஞ்சியத்தில் உள்ள அனைத்து சொற்களுக்கும் தொடர்புடைய வெக்டர்களை எடுக்கலாம்:


In [ ]:
vectors = torch.stack([embedder(torch.tensor(vocab[s])) for s in vocab.itos], 0)

வாங்க, உதாரணமாக **பாரிஸ்** என்ற வார்த்தை ஒரு வெக்டராக குறியாக்கப்படுவது எப்படி என்பதை பார்க்கலாம்:


In [ ]:
paris_vec = embedder(torch.tensor(vocab['paris']))
print(paris_vec)

tensor([-0.0915,  2.1224, -0.0281, -0.6819,  1.1219,  0.6458, -1.3704, -1.3314,
        -1.1437,  0.4496,  0.2301, -0.3515, -0.8485,  1.0481,  0.4386, -0.8949,
         0.5644,  1.0939, -2.5096,  3.2949, -0.2601, -0.8640,  0.1421, -0.0804,
        -0.5083, -1.0560,  0.9753, -0.5949, -1.6046,  0.5774],
       grad_fn=<EmbeddingBackward>)


Word2Vec-ஐ உபயோகித்து ஒத்த சொற்களை தேடுவது 흥미கரமாக உள்ளது. கீழே கொடுக்கப்பட்டுள்ள செயல்பாடு ஒரு குறிப்பிட்ட உள்ளீட்டு சொற்களுக்கு `n` மிக அருகிலுள்ள சொற்களை திருப்பும். அவற்றை கண்டுபிடிக்க, $|w_i - v|$-இன் நார்மை கணக்கிடுகிறோம், இங்கு $v$ என்பது நம் உள்ளீட்டு சொற்க்கு தொடர்பான வெக்டர், மற்றும் $w_i$ என்பது அகராதியில் உள்ள $i$-வது சொற்களின் குறியாக்கம். பின்னர், வரிசைப்படுத்தப்பட்ட வரிசையை `argsort` பயன்படுத்தி திருப்புகிறோம், மற்றும் அகராதியில் மிக அருகிலுள்ள சொற்களின் இடங்களை குறிக்கும் பட்டியலின் முதல் `n` உருப்படிகளை எடுக்கிறோம்.


In [ ]:
def close_words(x, n = 5):
  vec = embedder(torch.tensor(vocab[x]))
  top5 = np.linalg.norm(vectors.detach().numpy() - vec.detach().numpy(), axis = 1).argsort()[:n]
  return [ vocab.itos[x] for x in top5 ]

close_words('microsoft')

['microsoft', 'quoted', 'lp', 'rate', 'top']

In [ ]:
close_words('basketball')

['basketball', 'lot', 'sinai', 'states', 'healthdaynews']

In [ ]:
close_words('funds')

['funds', 'travel', 'sydney', 'japan', 'business']

## முக்கிய குறிப்புகள்

CBoW போன்ற புத்திசாலி நுட்பங்களைப் பயன்படுத்தி, Word2Vec மாதிரியை பயிற்றுவிக்கலாம். மையச் சொற்களை வழங்கி அதற்குச் சுற்றியுள்ள சொற்களை கணிக்க பயிற்றுவிக்கப்படும் skip-gram மாதிரியை நீங்கள் முயற்சித்து, அது எவ்வளவு சிறப்பாக செயல்படுகிறது என்பதைப் பாருங்கள்.



---

**குறிப்பு**:  
இந்த ஆவணம் [Co-op Translator](https://github.com/Azure/co-op-translator) என்ற AI மொழிபெயர்ப்பு சேவையைப் பயன்படுத்தி மொழிபெயர்க்கப்பட்டுள்ளது. நாங்கள் துல்லியத்திற்காக முயற்சிக்கின்றோம், ஆனால் தானியங்கி மொழிபெயர்ப்புகளில் பிழைகள் அல்லது தவறான தகவல்கள் இருக்கக்கூடும் என்பதை தயவுசெய்து கவனத்தில் கொள்ளவும். அதன் தாய்மொழியில் உள்ள மூல ஆவணம் அதிகாரப்பூர்வ ஆதாரமாக கருதப்பட வேண்டும். முக்கியமான தகவல்களுக்கு, தொழில்முறை மனித மொழிபெயர்ப்பு பரிந்துரைக்கப்படுகிறது. இந்த மொழிபெயர்ப்பைப் பயன்படுத்துவதால் ஏற்படும் எந்த தவறான புரிதல்கள் அல்லது தவறான விளக்கங்களுக்கு நாங்கள் பொறுப்பல்ல.
